In [ ]:
import os
os.environ["JAVA_HOME"] = "D:\jdk"
os.environ["SPARK_HOME"] = "D:\spark"
import findspark
findspark.init()
from pyspark import SparkConf, SparkContext
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.feature import IDF
sparkcon = SparkContext.getOrCreate()

In [ ]:
# Data loading
dfRaw = sparkcon.textFile("small.tsv")
fields = dfRaw.map(lambda x: x.split("\t"))
dfs = fields.map(lambda x: x[3].split(" "))
dfID = fields.map(lambda x: x[1])

# words frequencies:
hashingTF = HashingTF(100000)  #100K hash buckets
tf = hashingTF.transform(dfs)

# compute the TF*IDF in each document:
tf.cache()
idf = IDF(minDocFreq=2).fit(tf)
tfidf = idf.transform(tf)

In [7]:
#example: search for "Gettysburg":

gettysburgTF = hashingTF.transform(["Gettysburg"])
gettysburgHashValue = int(gettysburgTF.indices[0])

# extract the TF*IDF sparkconore 
gettysburgRelevance = tfidf.map(lambda x: x[gettysburgHashValue])

zippedResults = gettysburgRelevance.zip(dfID)

# print the document with the maximum TF*IDF value:
print("maximum TF*IDF value for Gettysburg is:")
print(zippedResults.max())

maximum TF*IDF value for Gettysburg is:
(0.0, 'Ælle of Sussex')
